In [1]:
!pip install pyLDAvis

  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blis 1.0.1 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [51]:
!pip install spacy

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.0.2 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/11.8 MB ? eta -:--:--
    --------------------------------------- 0.3/11.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.8 MB 2.4 MB/s eta 0:00:05
   - -------------------------------------- 0.5/11.8 MB 2.4 MB/s eta 0:00:05
   -- ------------------------------------- 0.8/11.8 MB 838.9 kB/s eta 0:00:14
   -- ------------------------------------- 0.8/11.8 MB 838.9 kB/s eta 0:00:14
   --- ------------------------------------ 1.0/11.8 MB 868.0 kB/s eta 0:00:13
   --- ------------------------------------ 1.0/11.8 MB 868.0 kB/s eta 0:00:13
   ---- ----------------------------------- 1.3/11.8 MB 754.4 kB/s eta 0:00:14
   ----- ---------------------------------- 1.6/11.8 MB 806.6 kB/s eta 0:00:13
   ------ --------------------------------- 1.8/11.8 MB 868.0 kB/s eta 0:00:12
   ------ --------------------------------- 1.8/11.8 MB 868.0 kB/s eta 0:00:12
   ------ --------------------------------- 1.8/11.8 MB 868.0 kB/s eta 0:

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import os

filename = os.path.join(os.getcwd(), "data", "description_extracted.csv")
df = pd.read_csv(filename, header=0, encoding='utf-8')
df = df.dropna(subset=['ExtractedDescription'])
df.head()

,RequisitionID,ExtractedDescription
0,00000224907,**The Boeing Company** is in search of a **L...
1,00000331804,Find your future with us.\n\nJob Description\n...
2,00000336462,Find your future with us.\n\nWe are Boeing Res...
3,00000338951,In this role you will have an opportunity to w...
4,00000339390,The stress analyst must also be able to effect...


In [2]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')).union({'engineering', 'san francisco','skills', 'experience', 'work', 'team'})
vectorizer = CountVectorizer(stop_words=stop_words)

In [3]:
df['ExtractedDescription'] = df['ExtractedDescription'].str.replace(r'[^\w\s]', '', regex=True)  # Remove special chars

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
stop_words = list(stopwords.words('english')) + ['engineering', 'skills', 'experience', 'work', 'team']

vectorizer = TfidfVectorizer(
    max_features=500,
    max_df=0.9,  # Ignore common words
    min_df=5, # Ignore rare words
    stop_words=stop_words,
    ngram_range=(1, 2)  # for bigrams
)
dtm = vectorizer.fit_transform(df['ExtractedDescription'])

### Method 1

In [5]:
lda_model = LatentDirichletAllocation(
    n_components=5,  
    max_iter=20,      
    learning_decay=0.5,
    random_state=42,
)
lda_model.fit(dtm)

LatentDirichletAllocation(learning_decay=0.5, max_iter=20, n_components=5,
                          random_state=42)

In [6]:
#Visualize with pyLDAvis
import pyLDAvis.lda_model

pyLDAvis.enable_notebook()
lda_visualization = pyLDAvis.lda_model.prepare(lda_model, dtm, vectorizer)
pyLDAvis.display(lda_visualization)

In [7]:
feature_names = vectorizer.get_feature_names_out()

def get_top_keywords(model, feature_names, n_top_words):
    keywords = []
    for topic_idx, topic in enumerate(model.components_):
        keywords.append([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
    return keywords

top_keywords = get_top_keywords(lda_model, feature_names, 20)

for idx, keywords in enumerate(top_keywords):
    print(f"Cluster {idx + 1}: {', '.join(keywords)}")

Cluster 1: manufacturing, equipment, design, process, systems, control, power, support, project, technical, electrical, energy, quality, product, system, development, safety, knowledge, production, ability
Cluster 2: benefits, insurance, paid, career, us, employees, business, 401k, dental, also, people, leave, life, medical, performing, data, dental vision, time, disability, employer
Cluster 3: systems, test, security, design, development, technical, software, system, data, support, solutions, network, requirements, analysis, years, integration, hardware, management, technologies, knowledge
Cluster 4: project, construction, design, projects, technical, knowledge, ability, field, management, including, staff, plans, position, client, professional, engineer, water, clients, reports, civil
Cluster 5: apple, equipment, san, maintenance, repair, hvac, repairs, plumbing, building, pay, systems, base, required, applicable, guidelines, electrical, participates, law, maintains, locations


### Samiul's analysis

In [10]:
# Get topic distribution for each job description
topic_distributions = lda_model.transform(dtm)

# Assign the topic with the highest probability
df['Assigned_Topic'] = topic_distributions.argmax(axis=1)

# Print sample rows to verify
print(df[['ExtractedDescription', 'Assigned_Topic']].head())

                                ExtractedDescription  Assigned_Topic
0  The Boeing Company  is in search of a  Lead Gu...               2
1  Find your future with us\n\nJob Description\n\...               2
2  Find your future with us\n\nWe are Boeing Rese...               2
3  In this role you will have an opportunity to w...               2
4  The stress analyst must also be able to effect...               3


In [11]:
from collections import Counter

def suggest_lda_titles(df, text_column, topic_column, n_words=5):
    cluster_titles = {}
    for topic in df[topic_column].unique():
        topic_texts = df[df[topic_column] == topic][text_column]
        words = ' '.join(topic_texts).lower().split()
        word_counts = Counter(word for word in words if word not in stop_words and len(word) > 3)
        top_words = [word for word, _ in word_counts.most_common(n_words)]
        cluster_titles[topic] = ' '.join(top_words)
    return cluster_titles

# Generate titles for each LDA topic
lda_titles = suggest_lda_titles(df, 'ExtractedDescription', 'Assigned_Topic')

# Add to dataframe
df['Suggested_LDA_Title'] = df['Assigned_Topic'].map(lda_titles)

# Print suggested titles
print(lda_titles)

{2: 'systems design technical development support', 3: 'project design construction projects technical', 0: 'design systems equipment support technical', 1: 'data business solutions management tools', 4: 'equipment maintenance systems required repair'}


### Suggesting job titles using openai

In [ ]:
import openai

client = openai.OpenAI(api_key = "API_KEY")
def call_openai_api(cluster, keywords): 
    prompt = f"""
    Given the following job clusters and their top keywords extracted through a TF-IDF vectorizer and topic modeling 
    analysis using LDA, suggest 3 suitable job titles for each cluster that align with the identified themes. Return 
    this in JSON formatting. The format should be as follows:

    "Cluster n: title1, title2, title3"

    The cluster, along with their keywords, are as follows:
    {cluster}: {keywords}
    """
    try: 
        chat_completion = client.chat.completions.create( 
        messages=[ { 
            "role": "user", "content": "follow the format and DON't other weird delimiters." + prompt, 
        } ], 
        model="gpt-4o-mini", 
        )
        parsed_response =  chat_completion.choices[0].message.content
        return parsed_response
    
    except Exception as e: 
        return f"Error: {str(e)}"


In [22]:
import json

for idx, keywords in enumerate(top_keywords):
    titles = (call_openai_api(f"Cluster {idx + 1}",{', '.join(keywords)} ))
    try:
        parsed_json = json.loads(titles)
        formatted_json = json.dumps(parsed_json, indent=4)
        print(formatted_json)
    except:
        print(titles)

```json
{
    "Cluster 1": "Manufacturing Engineer, Electrical Equipment Designer, Process Control Specialist"
}
```
```json
{
    "Cluster 2": "Benefits Coordinator, Employee Relations Specialist, Human Resources Generalist"
}
```
```json
{
    "Cluster 3": "Software Development Engineer, Systems Security Analyst, Technical Support Engineer"
}
```
```json
{
    "Cluster 4": "Construction Project Manager, Civil Engineering Project Coordinator, Technical Design Engineer"
}
```
```json
{
  "Cluster 5": "HVAC Maintenance Technician, Equipment Repair Specialist, Building Systems Technician"
}
```
